In [1]:
import numpy as np
import pandas as pd
from Tools.scripts.make_ctype import values

from src.turboprop import Turboprop
from src.turbofan import Turbofan

config_ex71 = {
    "mach": 0.0,
    "t_a": 288.15,
    "p_a": 101.3,

    # Eficiências e Gammas
    "eta_inlet": 0.85,
    "gamma_inlet": 1.4,
    "eta_compressor": 0.75,
    "gamma_compressor": 1.37,
    "eta_camara": 1,
    "gamma_camara": 1.35,
    "eta_turbina_compressor": 0.9,
    "gamma_turbina_compressor": 1.33,
    "eta_turbina_livre": 0.9,
    "gamma_turbina_livre": 1.33,
    "eta_bocal_quente": 0.98,
    "gamma_bocal_quente": 1.36,
    "eta_propeler": 0.85,

    # Dados operacionais
    "max_gearbox_power": 2050.0,  # kW
    "prc": 15.77,
    # Ajusta-se pr_tl para se atingir a potência termodinâmica informada pelo fabricante
    "pr_tl": 2.87,
    "hydrogen_fraction": 0.0,
    "pressure_loss_factor": 1.0,
    "kerosene_PCI": 45e3,  # kJ/kg
    "hydrogen_PCI": 120e3,  # kJ/kg
    "mean_R_air": 288.3,  # (m^2 / (s^2*K))
    "Cp": 1.11,  # (kJ / (kg*K))
    "Cp_tl": 1.16,  # (kJ / (kg*K))
    "T04": 1600,  # (K)
    "power_ratio_gear_to_tl": 0.98,
    # Dados da gearbox e hélice
    "gearbox_efficiency": 0.98,  # potência que chega na hélice / potência que sai da turbina
    "propeller_efficiency": 0.85,
    "ref_pot_th": 2456.49,  # kW -> se não fornecido, pode ser considerado como 0.8 * Pot_th (termodinâmica)
}

config_ex72 = {
    "mach": 0.45,
    "t_a": 246.55,
    "p_a": 41.0,

    # Eficiências e Gammas
    "eta_inlet": 0.85,
    "gamma_inlet": 1.4,
    "eta_compressor": 0.75,
    "gamma_compressor": 1.37,
    "eta_camara": 1,
    "gamma_camara": 1.35,
    "eta_turbina_compressor": 0.9,
    "gamma_turbina_compressor": 1.33,
    "eta_turbina_livre": 0.9,
    "gamma_turbina_livre": 1.33,
    "eta_bocal_quente": 0.98,
    "gamma_bocal_quente": 1.36,
    "eta_propeler": 0.85,

    # Dados operacionais
    "max_gearbox_power": 2050.0,  # kW
    "prc": 15.77,
    # Ajusta-se pr_tl para se atingir a potência termodinâmica informada pelo fabricante
    "pr_tl": 2.87,
    "hydrogen_fraction": 0.0,
    "pressure_loss_factor": 1.0,
    "kerosene_PCI": 45e3,  # kJ/kg
    "hydrogen_PCI": 120e3,  # kJ/kg
    "mean_R_air": 288.3,  # (m^2 / (s^2*K))
    "Cp": 1.11,  # (kJ / (kg*K))
    "Cp_tl": 1.16,  # (kJ / (kg*K))
    "T04": 1600,  # (K)
    "power_ratio_gear_to_tl": 0.98,
    # Dados da gearbox e hélice
    "gearbox_efficiency": 0.98,  # potência que chega na hélice / potência que sai da turbina
    "propeller_efficiency": 0.85,
    "ref_pot_th": 2456.49,  # kW -> se não fornecido, pode ser considerado como 0.8 * Pot_th (termodinâmica)
}

config_ex73 = {
    "mach": 0.45,
    "altitude": 7000 * 0.3048,
    "t_a": 246.55,
    "p_a": 41.0,

    # Eficiências e Gammas
    "eta_inlet": 0.97,
    "gamma_inlet": 1.4,
    "eta_fan": 0.9,
    "gamma_fan": 1.4,
    "eta_compressor": 0.75,
    "gamma_compressor": 1.37,
    "eta_camara": 0.9995,
    "gamma_camara": 1.35,
    "eta_turbina_compressor": 0.9,
    "gamma_turbina_compressor": 1.33,
    "eta_turbina_fan": 0.9,
    "gamma_turbina_fan": 1.33,
    "eta_bocal_quente": 0.98,
    "gamma_bocal_quente": 1.36,
    "eta_bocal_fan": 0.98,
    "gamma_bocal_fan": 1.4,

    # Dados operacionais
    "bpr": 4.2,
    "prf": 1.5,
    "prc": 8.33,
    "hydrogen_fraction": 0.0,
    "pressure_loss_factor": 1.0,
    "kerosene_PCI": 45e3,  # kJ/kg
    "hydrogen_PCI": 120e3,  # kJ/kg
    "mean_R_air": 288.3,  # (m^2 / (s^2*K))
    "Cp": 1.11,  # (kJ / (kg*K))
    "T04": 1550,  # (K)
}


#define FLAG_LOWER  0x01
#define FLAG_UPPER  0x02
#define FLAG_ALPHA  (FLAG_LOWER|FLAG_UPPER)
#define FLAG_DIGIT  0x04
#define FLAG_ALNUM  (FLAG_ALPHA|FLAG_DIGIT)
#define FLAG_SPACE  0x08
#define FLAG_XDIGIT 0x10

static unsigned int ctype_table[256] = {
    0, /* 0x0 '\x00' */
    0, /* 0x1 '\x01' */
    0, /* 0x2 '\x02' */
    0, /* 0x3 '\x03' */
    0, /* 0x4 '\x04' */
    0, /* 0x5 '\x05' */
    0, /* 0x6 '\x06' */
    0, /* 0x7 '\x07' */
    0, /* 0x8 '\x08' */
    FLAG_SPACE, /* 0x9 '\t' */
    FLAG_SPACE, /* 0xa '\n' */
    FLAG_SPACE, /* 0xb '\v' */
    FLAG_SPACE, /* 0xc '\f' */
    FLAG_SPACE, /* 0xd '\r' */
    0, /* 0xe '\x0e' */
    0, /* 0xf '\x0f' */
    0, /* 0x10 '\x10' */
    0, /* 0x11 '\x11' */
    0, /* 0x12 '\x12' */
    0, /* 0x13 '\x13' */
    0, /* 0x14 '\x14' */
    0, /* 0x15 '\x15' */
    0, /* 0x16 '\x16' */
    0, /* 0x17 '\x17' */
    0, /* 0x18 '\x18' */
    0, /* 0x19 '\x19' */
    0, /* 0x1a '\x1a' */
    0, /* 0x1b '\x1b' */
    FLAG_SPACE, /* 0x1c '

In [2]:
turboprop = Turboprop(config_ex71)
turboprop.set_air_flow(8.49)
turboprop.save_design_point()
turboprop.calibrate_pot_th()
turboprop.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação              | Temp. Total (K)      | Pressão Total (kPa) 
------------------------------------------------------------------
2 (Inlet)            | 288.150              | 101.300             
3 (Compressor)       | 688.131              | 1446.661            
4 (Câmara)           | 1491.586             | 1446.661            
5 (Turbina Comp.)    | 1091.606             | 343.534             
6 (Turbina Livre)    | 863.409              | 119.402             

[ Dados da Turbina Livre ]
Pressão total na saída da tl       : 119.40 K
Temperatura Isentrópica Total      : 839.83 K
Temperatura Total                  : 863.41 K
Cp_tl                              : 1.16 K
Trabalho Isentrópico               : 292.06 kJ/kg
Trabalho Real                      : 264.71 kJ/kg
Potência Termodinâmica             : 2050.00 kW
Potência na Turbina Livre          : 2050.00 kW
Potência na Gearbox                : 2009.00 kW

[ Empuxo ]
E

C:\Users\andre\OneDrive\Área de Trabalho\TG\Python\src\turboprop.py:394: RuntimeWarning: divide by zero encountered in scalar divide
  thrust_kN = self.propeller_efficiency * self.pot_gear / self.u_flight


In [3]:
# turboprop = Turboprop(config_ex72)
# turboprop.set_sea_level_air_flow(8.49)
# turboprop.save_design_point()
# turboprop.update_from_N2(0.85)

In [4]:
turboprop.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação              | Temp. Total (K)      | Pressão Total (kPa) 
------------------------------------------------------------------
2 (Inlet)            | 256.535              | 46.156              
3 (Compressor)       | 577.216              | 534.552             
4 (Câmara)           | 1310.669             | 534.552             
5 (Turbina Comp.)    | 989.988              | 144.472             
6 (Turbina Livre)    | 798.767              | 55.869              

[ Dados da Turbina Livre ]
Pressão total na saída da tl       : 55.87 K
Temperatura Isentrópica Total      : 782.08 K
Temperatura Total                  : 798.77 K
Cp_tl                              : 1.16 K
Trabalho Isentrópico               : 241.17 kJ/kg
Trabalho Real                      : 221.82 kJ/kg
Potência Termodinâmica             : 633.10 kW
Potência na Turbina Livre          : 633.10 kW
Potência na Gearbox                : 620.44 kW

[ Empuxo ]
Empux

In [17]:
turbofan = Turbofan(config_ex73)
turbofan.set_sea_level_air_flow(72.26)
turbofan.save_design_point()
turbofan.update_from_N2(0.85)
turbofan.print_outputs()

0.7993385
Parâmetros atualizados para N2/N2_design = 0.8500
--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 256.535              | 46.920              
8 (Fan)         | 278.681              | 61.318              
3 (Compressor)  | 513.066              | 375.115             
4 (Câmara)      | 1269.711             | 375.115             
5 (Turbina Comp.) | 1035.326             | 145.703             
6 (Turbina Fan) | 901.120              | 76.415              

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 141.955 m/s
Velocidade Bocal Quente (u_core): 540.630 m/s
Velocidade Bocal Frio (u_fan)   : 244.695 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01932
Empuxo Específico               : 0.929 kN/(kg/s)
Consumo Específico (TSFC)       : 20.79247 kg/(s*kN)
Empuxo Total                    : 4.260 kN
Con